In [1]:
%cd /home/root2

/home/root2


In [2]:
!ls

AdvSemiSeg				Notebooks   save
CCT					data	    tmux-client-32898.log
Miniconda3-py37_4.12.0-Linux-x86_64.sh	miniconda3  tmux-server-32900.log


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

torch.manual_seed(42)

In [4]:
from torchvision import datasets, transforms

transform=transforms.Compose([
        # Pad images with 0s
        transforms.Pad((0,4,4,0), fill=0, padding_mode='constant'),
    
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        ])
dataset_full = datasets.MNIST('./data', train=True, download=True,
                   transform=transform)
valid_size = 5000
train_size = len(dataset_full) - 5000
dataset_train, dataset_valid = torch.utils.data.random_split(dataset_full, [train_size, valid_size], generator=torch.Generator().manual_seed(42))

dataset_test = datasets.MNIST('./data', train=False,
                   transform=transform)

train_kwargs = {'batch_size': 128, 'shuffle': True}
valid_kwargs = {'batch_size': 128, 'shuffle': False}
test_kwargs = {'batch_size': 128, 'shuffle': False}
train_loader = torch.utils.data.DataLoader(dataset_train,**train_kwargs)
valid_loader = torch.utils.data.DataLoader(dataset_valid,**valid_kwargs)
test_loader = torch.utils.data.DataLoader(dataset_test, **test_kwargs)

print()
print("Image Shape: {}".format(dataset_train[0][0].shape))
print()
print("Training Set:   {} samples".format(len(dataset_train)))
print("Validation Set:   {} samples".format(len(dataset_valid)))
print("Test Set:       {} samples".format(len(dataset_test)))


Image Shape: torch.Size([1, 32, 32])

Training Set:   55000 samples
Validation Set:   5000 samples
Test Set:       10000 samples


In [5]:
# load classifier
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, (5,5)),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, (5,5)),
            nn.BatchNorm2d(16),
            nn.ReLU(), 
            nn.MaxPool2d(2,2)
        )
        self.fc1   = nn.Linear(400, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features 

In [6]:
# Train & test part from https://github.com/activatedgeek/LeNet-5
def train(epoch):
    global cur_batch_win
    net.train()
    loss_list, batch_list = [], []
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        output = net(images.cuda())
        loss = criterion(output, labels.cuda())

        loss_list.append(loss.detach().cpu().item())
        batch_list.append(i+1)

        if i % 10 == 0:
            print('Train - Epoch %d, Batch: %d, Loss: %f' % (epoch, i, loss.detach().cpu().item()))

        loss.backward()
        optimizer.step()

In [7]:
net = LeNet().cuda()
print (net)

learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr=learning_rate)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
def evaluate(target_loader, target_dataset):
    predictions = []
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(target_loader):
        output = net(images.cuda())
        avg_loss += criterion(output, labels.cuda()).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.cuda().view_as(pred)).sum()
        predictions.append(pred)

    avg_loss /= len(target_dataset)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(target_dataset)))
    accuracy    = float(total_correct) / len(target_dataset)
    return accuracy, np.array(torch.cat(predictions).cpu())
    #or if you are in latest Pytorch world
    #return accuracy, np.array(torch.vstack(predictions))

In [9]:
print("Training...")
print()
best_val_acc = 0

for e in range(1, 200):
    train(e)
    validation_accuracy, validation_predictions = evaluate(valid_loader, dataset_valid)
    print("EPOCH {} ...".format(e))
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))
    print()
    
    if validation_accuracy > best_val_acc:
        validation_accuracy = best_val_acc

        torch.save(
            {
                'lenet': net.state_dict(),
            },
            ('./save/kiet_mnist_classifier.pt'),
        )
        print("Model saved")

Training...

Train - Epoch 1, Batch: 0, Loss: 2.305326
Train - Epoch 1, Batch: 10, Loss: 1.801166
Train - Epoch 1, Batch: 20, Loss: 1.173530
Train - Epoch 1, Batch: 30, Loss: 0.818145
Train - Epoch 1, Batch: 40, Loss: 0.450485
Train - Epoch 1, Batch: 50, Loss: 0.447638
Train - Epoch 1, Batch: 60, Loss: 0.375104
Train - Epoch 1, Batch: 70, Loss: 0.290163
Train - Epoch 1, Batch: 80, Loss: 0.172211
Train - Epoch 1, Batch: 90, Loss: 0.342018
Train - Epoch 1, Batch: 100, Loss: 0.191067
Train - Epoch 1, Batch: 110, Loss: 0.170591
Train - Epoch 1, Batch: 120, Loss: 0.200742
Train - Epoch 1, Batch: 130, Loss: 0.139756
Train - Epoch 1, Batch: 140, Loss: 0.112938
Train - Epoch 1, Batch: 150, Loss: 0.093955
Train - Epoch 1, Batch: 160, Loss: 0.254051
Train - Epoch 1, Batch: 170, Loss: 0.140887
Train - Epoch 1, Batch: 180, Loss: 0.223995
Train - Epoch 1, Batch: 190, Loss: 0.127031
Train - Epoch 1, Batch: 200, Loss: 0.225062
Train - Epoch 1, Batch: 210, Loss: 0.140406
Train - Epoch 1, Batch: 220, L

Train - Epoch 5, Batch: 70, Loss: 0.012586
Train - Epoch 5, Batch: 80, Loss: 0.006953
Train - Epoch 5, Batch: 90, Loss: 0.002565
Train - Epoch 5, Batch: 100, Loss: 0.029601
Train - Epoch 5, Batch: 110, Loss: 0.007108
Train - Epoch 5, Batch: 120, Loss: 0.040104
Train - Epoch 5, Batch: 130, Loss: 0.029370
Train - Epoch 5, Batch: 140, Loss: 0.006945
Train - Epoch 5, Batch: 150, Loss: 0.016986
Train - Epoch 5, Batch: 160, Loss: 0.032072
Train - Epoch 5, Batch: 170, Loss: 0.044288
Train - Epoch 5, Batch: 180, Loss: 0.101612
Train - Epoch 5, Batch: 190, Loss: 0.030361
Train - Epoch 5, Batch: 200, Loss: 0.013452
Train - Epoch 5, Batch: 210, Loss: 0.003021
Train - Epoch 5, Batch: 220, Loss: 0.006277
Train - Epoch 5, Batch: 230, Loss: 0.041248
Train - Epoch 5, Batch: 240, Loss: 0.024702
Train - Epoch 5, Batch: 250, Loss: 0.002762
Train - Epoch 5, Batch: 260, Loss: 0.063514
Train - Epoch 5, Batch: 270, Loss: 0.016360
Train - Epoch 5, Batch: 280, Loss: 0.030750
Train - Epoch 5, Batch: 290, Loss: 

Train - Epoch 9, Batch: 140, Loss: 0.001788
Train - Epoch 9, Batch: 150, Loss: 0.004155
Train - Epoch 9, Batch: 160, Loss: 0.007682
Train - Epoch 9, Batch: 170, Loss: 0.005256
Train - Epoch 9, Batch: 180, Loss: 0.008544
Train - Epoch 9, Batch: 190, Loss: 0.006473
Train - Epoch 9, Batch: 200, Loss: 0.019357
Train - Epoch 9, Batch: 210, Loss: 0.035918
Train - Epoch 9, Batch: 220, Loss: 0.016677
Train - Epoch 9, Batch: 230, Loss: 0.057461
Train - Epoch 9, Batch: 240, Loss: 0.002089
Train - Epoch 9, Batch: 250, Loss: 0.015452
Train - Epoch 9, Batch: 260, Loss: 0.000634
Train - Epoch 9, Batch: 270, Loss: 0.068476
Train - Epoch 9, Batch: 280, Loss: 0.025930
Train - Epoch 9, Batch: 290, Loss: 0.013122
Train - Epoch 9, Batch: 300, Loss: 0.016229
Train - Epoch 9, Batch: 310, Loss: 0.018616
Train - Epoch 9, Batch: 320, Loss: 0.001479
Train - Epoch 9, Batch: 330, Loss: 0.013950
Train - Epoch 9, Batch: 340, Loss: 0.012650
Train - Epoch 9, Batch: 350, Loss: 0.010134
Train - Epoch 9, Batch: 360, Los

KeyboardInterrupt: 

In [10]:
checkpoint = torch.load('./save/kiet_mnist_classifier.pt')
net.load_state_dict(checkpoint['lenet'])

test_accuracy, test_predictions = evaluate(test_loader, dataset_test)
print("Test Accuracy = {:.3f}".format(test_accuracy))

Test Avg. Loss: 0.000265, Accuracy: 0.989900
Test Accuracy = 0.990
